# Explore here

In [2]:
url = 'https://storage.googleapis.com/datascience-materials/dogs-vs-cats.zip'

import requests
import zipfile
from io import BytesIO

extract_to_directory = "extracted_content" # Directory to extract files into

# Send GET request and wrap content in BytesIO
response = requests.get(url)
with zipfile.ZipFile(BytesIO(response.content)) as zfile:
    # Extract all files to a specified directory
    zfile.extractall(extract_to_directory)

print(f"Content extracted to '{extract_to_directory}' successfully.")

Content extracted to 'extracted_content' successfully.
